In [2]:
# Librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score)
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
project_path = "../bridge_project"
df_original = pd.read_csv(f"{project_path}/df1_original_cod.csv")
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   turns                  20058 non-null  int64
 1   white_rating           20058 non-null  int64
 2   black_rating           20058 non-null  int64
 3   opening_moves          20058 non-null  int64
 4   victory_status_cod     20058 non-null  int64
 5   winner_cod             20058 non-null  int64
 6   time_increment_cod     20058 non-null  int64
 7   opening_code_cod       20058 non-null  int64
 8   opening_fullname_cod   20058 non-null  int64
 9   opening_shortname_cod  20058 non-null  int64
 10  opening_variation_cod  20058 non-null  int64
 11  rated_cod              20058 non-null  int64
dtypes: int64(12)
memory usage: 1.8 MB


In [4]:
def eval_perform(Y,Yhat, train_or_test):
    """Evalúa el performance de cada modelo

    Args:
        Y (array): Variable objetivo original
        Yhat (array): Salida Y & "hat" (ŷ) que denota predicciones estimadas.
        train_or_test (string): "Entrenamiento" o "Prueba"
    """
    accu = accuracy_score(Y,Yhat)
    prec = precision_score(Y,Yhat,average='weighted')
    reca = recall_score(Y,Yhat,average='weighted')
    print(f"\nPerformance del modelo de {train_or_test}")
    print('Accu \t Prec \t Reca\n Eval \t %0.3f \t %0.3f \t %0.3f'%(accu,prec,reca))

# Df original codificado y sin user_id White_id, Black_id y Opening_response (Variables no relevantes o con muchos valores nulos). 

In [5]:
df = df_original.copy()
y_name = "winner_cod"
# X es el dataframe eliminando la variable de salida
X = df.drop(columns=[y_name])
# Y es un array unidimensional (ravel) de la variable de salida
Y = df[y_name].ravel()


# Escalamiento de los datos
Después de correr la máquina de soporte vectorial y encontrar que tardaba horas se hizo una investigación con dos descubrimientos.

Cuando se tienen columnas con muchas clases, se incrementa el tiempo de ejecución por mucho esto se ve todavía más grabe con el Kernel lineal.
Por esta razon se aplica un escalamiento estandar y se logra disminuir el tiempo en el algorítmo más tardado (SVM con kernel lineal de 18 horas a XXX). 


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(df.drop(columns=[y_name])),columns = df.drop(columns=[y_name]).columns)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3)

# Maquina de soporte vectorial

In [8]:
# Crear un clasificador SVM para clasificación con kernel lineal, polinomial y de funcion de base radial
mod_linear = svm.SVC(kernel='linear',C=1)
mod_poly = svm.SVC(kernel='poly',degree=2,C=1)
mod_rbf = svm.SVC(kernel='rbf',C=1,gamma='auto')

In [9]:
# Entrenar el clasificador con los datos de entrenamiento
mod_linear.fit(X_train, Y_train)

SVC(C=1, kernel='linear')

In [10]:
mod_poly.fit(X_train, Y_train)

SVC(C=1, degree=2, kernel='poly')

In [11]:
mod_rbf.fit(X_train, Y_train)

SVC(C=1, gamma='auto')

In [12]:
# Salida Y & "hat" (ŷ) que denota predicciones estimadas.
Yhat_linear_test = mod_linear.predict(X_test)
Yhat_linear_train = mod_linear.predict(X_train)
eval_perform(Y_test,Yhat_linear_test, "Prueba")
eval_perform(Y_train,Yhat_linear_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.672 	 0.675 	 0.672

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.666 	 0.668 	 0.666


In [13]:
Yhat_poly_test = mod_poly.predict(X_test)
Yhat_poly_train = mod_poly.predict(X_train)
eval_perform(Y_test,Yhat_poly_test, "Prueba")
eval_perform(Y_train,Yhat_poly_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.568 	 0.589 	 0.568

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.569 	 0.595 	 0.569


In [14]:
Yhat_rbf_test = mod_rbf.predict(X_test)
Yhat_rbf_train = mod_rbf.predict(X_train)
eval_perform(Y_test,Yhat_rbf_test, "Prueba")
eval_perform(Y_train,Yhat_rbf_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.665 	 0.666 	 0.665

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.696 	 0.698 	 0.696


# Regresión logística

In [15]:
from sklearn.linear_model import LogisticRegression
# Comparacion con regresion logistica
# Usar regularización L1 (Lasso)
reg_log = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)

In [16]:
reg_log.fit(X_train,Y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [17]:
# Salida Y & "hat" (ŷ) que denota predicciones estimadas.
Yhat_log_test = reg_log.predict(X_test)
Yhat_log_train = reg_log.predict(X_train)
eval_perform(Y_test,Yhat_log_test, "Prueba")
eval_perform(Y_train,Yhat_log_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.673 	 0.675 	 0.673

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.664 	 0.665 	 0.664


# XGboost de ejemplo para comparar el performance (datos originales)

In [18]:
import xgboost as xgb
# Crear un clasificador XGBoost
xgboost = xgb.XGBClassifier()

In [20]:
# Entrenar el modelo en los datos de entrenamiento
xgboost.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [21]:
Yhat_xgboost_test = xgboost.predict(X_test)
Yhat_xgboost_train = xgboost.predict(X_train)
eval_perform(Y_test,Yhat_xgboost_test, "Prueba")
eval_perform(Y_train,Yhat_xgboost_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.883 	 0.883 	 0.883

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.950 	 0.950 	 0.950
